#Modelo para identificação de idiomas

##Bibliotecas

In [22]:
import pandas as pd
import re

from nltk.util import bigrams
from nltk.lm.preprocessing import pad_both_ends
from nltk.tokenize import WhitespaceTokenizer
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE
from nltk.lm import Laplace
from nltk.lm import NgramCounter

from sklearn.model_selection import train_test_split

#Dataset

In [2]:
dados_portugues = pd.read_csv("stackoverflow_portugues.csv")
dados_portugues.head()

,Id,Título,Questão,Tags,Pontuação,Visualizações
0,2402,Como fazer hash de senhas de forma segura?,"<p>Se eu fizer o <em><a href=""http://pt.wikipe...",<hash><segurança><senhas><criptografia>,350,22367
1,6441,Qual é a diferença entre INNER JOIN e OUTER JOIN?,<p>Qual é a diferença entre <code>INNER JOIN</...,<sql><join>,276,176953
2,579,Por que não devemos usar funções do tipo mysql_*?,<p>Uma dúvida muito comum é por que devemos pa...,<php><mysql>,226,9761
3,2539,As mensagens de erro devem se desculpar?,<p>É comum encontrar uma mensagem de erro que ...,<aplicação-web><gui><console><ux>,214,5075
4,17501,"Qual é a diferença de API, biblioteca e Framew...",<p>Me parecem termos muito próximos e eventual...,<api><framework><terminologia><biblioteca>,193,54191


In [3]:
dados_ingles = pd.read_csv("stackoverflow_ingles.csv")
dados_ingles.head()

,Id,Título,Questão,Tags,Pontuação,Visualizações
0,11227809,Why is it faster to process a sorted array tha...,<p>Here is a piece of C++ code that seems very...,<java><c++><performance><optimization><branch-...,23057,1358574
1,927358,How do I undo the most recent local commits in...,<p>I accidentally committed the wrong files to...,<git><version-control><git-commit><undo>,19640,7906137
2,2003505,How do I delete a Git branch locally and remot...,<p>I want to delete a branch both locally and ...,<git><git-branch><git-remote>,15249,6940906
3,292357,What is the difference between 'git pull' and ...,<blockquote>\n <p><strong>Moderator Note:</st...,<git><git-pull><git-fetch>,11008,2543052
4,477816,What is the correct JSON content type?,"<p>I've been messing around with <a href=""http...",<json><http-headers><content-type>,9701,2478940


In [4]:
dados_espanhol = pd.read_csv("stackoverflow_espanhol.csv", sep=';', encoding="latin1")
dados_espanhol.head()

,Id,Título,Questão,Tags,Pontuação,Visualizações
0,18232,¿Cómo evitar la inyección SQL en PHP?,<p>Las sentencias dinámicas son sentencias SQL...,<php><mysql><sql><seguridad><inyección-sql>,169,38614
1,197,¿Por qué mis programas no pueden hacer cálculo...,<p>Unas veces los cálculos funcionan correctam...,<matemáticas><coma-flotante><lenguaje-agnóstico>,141,3860
2,36,¿Cuál es la diferencia entre un inner y un out...,<p>¿Cuál es la diferencia entre un <code>inner...,<mysql><sql><join>,97,53627
3,29177,¿Por qué es considerado una mala práctica util...,"<p>La mayoría de nosotros decimos, (muchas vec...",<variables><variables-globales><patrones-de-di...,89,9987
4,142,Validar un email en JavaScript que acepte todo...,<h3>Pregunta</h3>\n\n<p>¿Cómo validar un e-mai...,<javascript><validación><email><regex>,87,73129


##Tratamento dos dados

In [5]:
regex_html = re.compile(r"<.*?>")
regex_codigo = re.compile(r"<code>(.|(\n))*?</code>")
regex_pontuacao = re.compile(r"[^\w\s]")
regex_digitos = re.compile(r"\d+")
regex_espaco = re.compile(r" +")
regex_quebra_linha = re.compile(r"(\n)")

In [6]:
def remover(textos, regex):
  if type(textos) == str:
    return regex.sub("", textos)
  else:
    return [regex.sub("", texto) for texto in textos]

In [7]:
def substituir_codigo(textos, regex):
  if type(textos) == str:
    return regex.sub("CODE", textos)
  else:
    return [regex.sub("CODE", texto) for texto in textos]

In [8]:
def minusculo(textos):
  if type(textos) == str:
    return textos.lower()
  else:
    return [texto.lower() for texto in textos]

In [9]:
def substituir_por_espaco(textos, regex):
  if type(textos) == str:
    return regex.sub(" ", textos)
  else:
    return [regex.sub(" ", texto) for texto in textos]

In [10]:
questoes_port_sem_code = substituir_codigo(dados_portugues.Questão, regex_codigo)
questoes_port_sem_code_tag = remover(questoes_port_sem_code, regex_html)
dados_portugues["sem_code_tag"] = questoes_port_sem_code_tag

questoes_port_sem_pont = remover(dados_portugues.sem_code_tag, regex_pontuacao)
questoes_port_sem_pont_minus = minusculo(questoes_port_sem_pont)
questoes_port_sem_pont_minus_dig = remover(questoes_port_sem_pont_minus, regex_digitos)
questoes_port_sem_quebra_linha = substituir_por_espaco(questoes_port_sem_pont_minus_dig, regex_quebra_linha)
questoes_port_sem_espaco_duplicado = substituir_por_espaco(questoes_port_sem_quebra_linha, regex_espaco)

dados_portugues["questoes_tratadas"] = questoes_port_sem_espaco_duplicado
dados_portugues.head()

,Id,Título,Questão,Tags,Pontuação,Visualizações,sem_code_tag,questoes_tratadas
0,2402,Como fazer hash de senhas de forma segura?,"<p>Se eu fizer o <em><a href=""http://pt.wikipe...",<hash><segurança><senhas><criptografia>,350,22367,Se eu fizer o hash de senhas antes de armazená...,se eu fizer o hash de senhas antes de armazená...
1,6441,Qual é a diferença entre INNER JOIN e OUTER JOIN?,<p>Qual é a diferença entre <code>INNER JOIN</...,<sql><join>,276,176953,Qual é a diferença entre CODE e CODE? Podem me...,qual é a diferença entre code e code podem me ...
2,579,Por que não devemos usar funções do tipo mysql_*?,<p>Uma dúvida muito comum é por que devemos pa...,<php><mysql>,226,9761,Uma dúvida muito comum é por que devemos parar...,uma dúvida muito comum é por que devemos parar...
3,2539,As mensagens de erro devem se desculpar?,<p>É comum encontrar uma mensagem de erro que ...,<aplicação-web><gui><console><ux>,214,5075,É comum encontrar uma mensagem de erro que diz...,é comum encontrar uma mensagem de erro que diz...
4,17501,"Qual é a diferença de API, biblioteca e Framew...",<p>Me parecem termos muito próximos e eventual...,<api><framework><terminologia><biblioteca>,193,54191,Me parecem termos muito próximos e eventualmen...,me parecem termos muito próximos e eventualmen...


In [11]:
questoes_ing_sem_code = substituir_codigo(dados_ingles.Questão, regex_codigo)
questoes_ing_sem_code_tag = remover(questoes_ing_sem_code, regex_html)
dados_ingles["sem_code_tag"] = questoes_ing_sem_code_tag

questoes_ing_sem_pont = remover(dados_ingles.sem_code_tag, regex_pontuacao)
questoes_ing_sem_pont_minus = minusculo(questoes_ing_sem_pont)
questoes_ing_sem_pont_minus_dig = remover(questoes_ing_sem_pont_minus, regex_digitos)
questoes_ing_sem_quebra_linha = substituir_por_espaco(questoes_ing_sem_pont_minus_dig, regex_quebra_linha)
questoes_ing_sem_espaco_duplicado = substituir_por_espaco(questoes_ing_sem_quebra_linha, regex_espaco)

dados_ingles["questoes_tratadas"] = questoes_ing_sem_espaco_duplicado
dados_ingles.head()

,Id,Título,Questão,Tags,Pontuação,Visualizações,sem_code_tag,questoes_tratadas
0,11227809,Why is it faster to process a sorted array tha...,<p>Here is a piece of C++ code that seems very...,<java><c++><performance><optimization><branch-...,23057,1358574,Here is a piece of C++ code that seems very pe...,here is a piece of c code that seems very pecu...
1,927358,How do I undo the most recent local commits in...,<p>I accidentally committed the wrong files to...,<git><version-control><git-commit><undo>,19640,7906137,I accidentally committed the wrong files to Gi...,i accidentally committed the wrong files to gi...
2,2003505,How do I delete a Git branch locally and remot...,<p>I want to delete a branch both locally and ...,<git><git-branch><git-remote>,15249,6940906,I want to delete a branch both locally and rem...,i want to delete a branch both locally and rem...
3,292357,What is the difference between 'git pull' and ...,<blockquote>\n <p><strong>Moderator Note:</st...,<git><git-pull><git-fetch>,11008,2543052,\n Moderator Note: Given that this question h...,moderator note given that this question has a...
4,477816,What is the correct JSON content type?,"<p>I've been messing around with <a href=""http...",<json><http-headers><content-type>,9701,2478940,I've been messing around with JSON for some ti...,ive been messing around with json for some tim...


In [12]:
questoes_esp_sem_code = substituir_codigo(dados_espanhol.Questão, regex_codigo)
questoes_esp_sem_code_tag = remover(questoes_esp_sem_code, regex_html)
dados_espanhol["sem_code_tag"] = questoes_esp_sem_code_tag

questoes_esp_sem_pont = remover(dados_espanhol.sem_code_tag, regex_pontuacao)
questoes_esp_sem_pont_minus = minusculo(questoes_esp_sem_pont)
questoes_esp_sem_pont_minus_dig = remover(questoes_esp_sem_pont_minus, regex_digitos)
questoes_esp_sem_quebra_linha = substituir_por_espaco(questoes_esp_sem_pont_minus_dig, regex_quebra_linha)
questoes_esp_sem_espaco_duplicado = substituir_por_espaco(questoes_esp_sem_quebra_linha, regex_espaco)

dados_espanhol["questoes_tratadas"] = questoes_esp_sem_espaco_duplicado
dados_espanhol.head()

,Id,Título,Questão,Tags,Pontuação,Visualizações,sem_code_tag,questoes_tratadas
0,18232,¿Cómo evitar la inyección SQL en PHP?,<p>Las sentencias dinámicas son sentencias SQL...,<php><mysql><sql><seguridad><inyección-sql>,169,38614,Las sentencias dinámicas son sentencias SQL qu...,las sentencias dinámicas son sentencias sql qu...
1,197,¿Por qué mis programas no pueden hacer cálculo...,<p>Unas veces los cálculos funcionan correctam...,<matemáticas><coma-flotante><lenguaje-agnóstico>,141,3860,Unas veces los cálculos funcionan correctament...,unas veces los cálculos funcionan correctament...
2,36,¿Cuál es la diferencia entre un inner y un out...,<p>¿Cuál es la diferencia entre un <code>inner...,<mysql><sql><join>,97,53627,¿Cuál es la diferencia entre un CODE y un CODE...,cuál es la diferencia entre un code y un code ...
3,29177,¿Por qué es considerado una mala práctica util...,"<p>La mayoría de nosotros decimos, (muchas vec...",<variables><variables-globales><patrones-de-di...,89,9987,"La mayoría de nosotros decimos, (muchas veces ...",la mayoría de nosotros decimos muchas veces si...
4,142,Validar un email en JavaScript que acepte todo...,<h3>Pregunta</h3>\n\n<p>¿Cómo validar un e-mai...,<javascript><validación><email><regex>,87,73129,Pregunta\n\n¿Cómo validar un e-mail que acepte...,pregunta cómo validar un email que acepte todo...


In [13]:
dados_portugues['idioma'] = 'port'
dados_ingles['idioma'] = 'ing'
dados_espanhol['idioma'] = 'esp'

##Criação do modelo de linguagem

In [14]:
port_treino, port_teste = train_test_split(dados_portugues.questoes_tratadas, test_size=0.2, random_state=123)

ing_treino, ing_teste = train_test_split(dados_ingles.questoes_tratadas, test_size=0.2, random_state=123)

esp_treino, esp_teste = train_test_split(dados_espanhol.questoes_tratadas, test_size=0.2, random_state=123)

In [15]:
todas_questoes_port = ' '.join(port_treino)

todas_palavras_port = WhitespaceTokenizer().tokenize(todas_questoes_port)

port_treino_bigram, vocab_port = padded_everygram_pipeline(2, todas_palavras_port)

In [16]:
def treinar_modelo_mle(lista_textos):
  todas_questoes = ' '.join(lista_textos)
  todas_palavras = WhitespaceTokenizer().tokenize(todas_questoes)
  bigrams, vocabulario = padded_everygram_pipeline(2, todas_palavras)
  modelo = MLE(2)
  modelo.fit(bigrams, vocabulario)

  return modelo

In [17]:
modelo_port = treinar_modelo_mle(port_treino)
modelo_ing = treinar_modelo_mle(ing_treino)
modelo_esp = treinar_modelo_mle(esp_treino)

In [18]:
def calcular_perplexidade(modelo, texto):
  perplexidade = 0
  palavras = WhitespaceTokenizer().tokenize(texto)
  palavras_fakechar = [list(pad_both_ends(palavra, n=2)) for palavra in palavras]
  palavras_bigramns = [list(bigrams(palavra)) for palavra in palavras_fakechar]

  for palavra in palavras_bigramns:
    perplexidade += modelo.perplexity(palavra)

  return perplexidade

In [19]:
print(calcular_perplexidade(modelo_port, port_teste.iloc[0]))

2006.9786364086417


In [21]:
print(calcular_perplexidade(modelo_ing, port_teste.iloc[0]))

inf


In [23]:
def treinar_modelo_Laplace(lista_textos):
  todas_questoes = ' '.join(lista_textos)
  todas_palavras = WhitespaceTokenizer().tokenize(todas_questoes)
  bigrams, vocabulario = padded_everygram_pipeline(2, todas_palavras)
  modelo = Laplace(2)
  modelo.fit(bigrams, vocabulario)

  return modelo

In [24]:
modelo_ing_Laplace = treinar_modelo_Laplace(ing_treino)
print(calcular_perplexidade(modelo_ing_Laplace, port_teste.iloc[0]))

5876.837588345698


In [25]:
modelo_port_Laplace = treinar_modelo_Laplace(port_treino)
print(calcular_perplexidade(modelo_port_Laplace, port_teste.iloc[0]))

2009.1937946178912


In [26]:
modelo_esp_Laplace = treinar_modelo_Laplace(esp_treino)
print(calcular_perplexidade(modelo_esp_Laplace, port_teste.iloc[0]))

3461.1385826250985


In [28]:
def atribui_idioma(lista_textos):

  idioma = []
  for texto in lista_textos:
    portugues = calcular_perplexidade(modelo_port_Laplace, texto)
    ingles = calcular_perplexidade(modelo_ing_Laplace, texto)
    espanhol = calcular_perplexidade(modelo_esp_Laplace, texto)

    if ingles >= portugues <= espanhol:
      idioma.append("portugues")
    elif portugues > ingles < espanhol:
      idioma.append("ingles")
    else:
      idioma.append("espanhol")
    
  return idioma

In [29]:
resultados_portugues = atribui_idioma(port_teste)
resultados_ingles = atribui_idioma(ing_teste)
resultados_espanhol = atribui_idioma(esp_teste)

In [30]:
print(resultados_portugues.count("portugues") / len(port_teste))
print(resultados_ingles.count("ingles") / len(ing_teste))
print(resultados_espanhol.count("espanhol") / len(esp_teste))

1.0
1.0
0.97
